In [7]:
import os
import pandas as pd

os.chdir(r'E:\ML in Fin Lab\group_proj\American-Options-Pricing-NN-Tree-models-LSMC-\LSMC American Option Pricing')
folder_path = r"data/spy_eod_202312.csv"
data = pd.read_csv(folder_path, low_memory=False)

In [9]:
data

,[QUOTE_UNIXTIME],[QUOTE_READTIME],[QUOTE_DATE],[QUOTE_TIME_HOURS],[UNDERLYING_LAST],[EXPIRE_DATE],[EXPIRE_UNIX],[DTE],[C_DELTA],[C_GAMMA],...,[P_LAST],[P_DELTA],[P_GAMMA],[P_VEGA],[P_THETA],[P_RHO],[P_IV],[P_VOLUME],[STRIKE_DISTANCE],[STRIKE_DISTANCE_PCT]
0,1701464400,2023-12-01 16:00,2023-12-01,16.0,459.02,2023-12-01,1701464400,0.0,1.00000,0.00000,...,0.01,0.00000,-0.00002,0.00055,-0.00494,0.00000,3.168950,0.000000,174.0,0.379
1,1701464400,2023-12-01 16:00,2023-12-01,16.0,459.02,2023-12-01,1701464400,0.0,1.00000,0.00000,...,0.02,-0.00070,0.00006,-0.00013,-0.00459,0.00000,3.059150,0.000000,169.0,0.368
2,1701464400,2023-12-01 16:00,2023-12-01,16.0,459.02,2023-12-01,1701464400,0.0,1.00000,0.00000,...,0.01,-0.00025,0.00000,0.00018,-0.00500,0.00000,2.951260,0.000000,164.0,0.357
3,1701464400,2023-12-01 16:00,2023-12-01,16.0,459.02,2023-12-01,1701464400,0.0,1.00000,0.00000,...,0.01,0.00000,0.00006,0.00060,-0.00509,0.00000,2.845330,1.000000,159.0,0.346
4,1701464400,2023-12-01 16:00,2023-12-01,16.0,459.02,2023-12-01,1701464400,0.0,1.00000,0.00000,...,0.01,-0.00032,-0.00001,0.00044,-0.00504,0.00000,2.740540,0.000000,154.0,0.336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79388,1703883600,2023-12-29 16:00,2023-12-29,16.0,475.31,2026-12-18,1797627600,1085.0,0.07062,0.00129,...,0.00,-0.90644,0.00332,0.81710,-0.00348,-1.61995,0.254010,,239.7,0.504
79389,1703883600,2023-12-29 16:00,2023-12-29,16.0,475.31,2026-12-18,1797627600,1085.0,0.06917,0.00127,...,0.00,-0.90539,0.00321,0.82843,-0.00379,-1.65202,0.258080,,244.7,0.515
79390,1703883600,2023-12-29 16:00,2023-12-29,16.0,475.31,2026-12-18,1797627600,1085.0,0.07519,0.00124,...,0.00,-0.90146,0.00311,0.84973,-0.00377,-1.71758,0.266230,,254.7,0.536
79391,1703883600,2023-12-29 16:00,2023-12-29,16.0,475.31,2026-12-18,1797627600,1085.0,0.05853,0.00110,...,0.00,-0.89835,0.00299,0.87259,-0.00433,-1.78361,0.273760,,264.7,0.557


In [10]:
data.columns = data.columns.map(lambda x: x[2:-1].lower())
data['r'] = 0.05

## Convert Columns to Numeric

In [11]:
# Check for non-numeric entries in relevant columns
columns_to_check = ['underlying_last', 'strike', 'dte', 'c_iv', 'p_iv']
for column in columns_to_check:
    print(f"Non-numeric values in {column}:")
    print(data[~data[column].apply(lambda x: isinstance(x, (int, float)))][column])

Non-numeric values in underlying_last:
Series([], Name: underlying_last, dtype: float64)
Non-numeric values in strike:
Series([], Name: strike, dtype: float64)
Non-numeric values in dte:
Series([], Name: dte, dtype: float64)
Non-numeric values in c_iv:
0                 
1                 
2                 
3                 
4                 
           ...    
79388     0.126020
79389     0.128220
79390     0.136300
79391     0.130120
79392     0.134830
Name: c_iv, Length: 79393, dtype: object
Non-numeric values in p_iv:
0         3.168950
1         3.059150
2         2.951260
3         2.845330
4         2.740540
           ...    
79388     0.254010
79389     0.258080
79390     0.266230
79391     0.273760
79392     0.281320
Name: p_iv, Length: 79393, dtype: object


In [12]:
columns_to_convert = ['c_iv', 'p_iv']
for column in columns_to_convert:
    data[column] = pd.to_numeric(data[column], errors='coerce')

## Handle Missing Values

In [14]:
print(data.isna().sum())

uote_unixtime             0
quote_readtime            0
quote_date                0
quote_time_hours          0
underlying_last           0
expire_date               0
expire_unix               0
dte                       0
c_delta                   0
c_gamma                   0
c_vega                    0
c_theta                   0
c_rho                     0
c_iv                   4098
c_volume                  0
c_last                    0
c_size                    0
c_bid                     0
c_ask                     0
strike                    0
p_bid                     0
p_ask                     0
p_size                    0
p_last                    0
p_delta                   0
p_gamma                   0
p_vega                    0
p_theta                   0
p_rho                     0
p_iv                   4497
p_volume                  0
strike_distance           0
strike_distance_pct       0
r                         0
dtype: int64


In [15]:
data = data.dropna(subset=columns_to_convert)
print(data.dtypes)
print(data[columns_to_check].head())

uote_unixtime            int64
quote_readtime          object
quote_date              object
quote_time_hours       float64
underlying_last        float64
expire_date             object
expire_unix              int64
dte                    float64
c_delta                float64
c_gamma                float64
c_vega                 float64
c_theta                float64
c_rho                  float64
c_iv                   float64
c_volume                object
c_last                 float64
c_size                  object
c_bid                  float64
c_ask                  float64
strike                 float64
p_bid                  float64
p_ask                  float64
p_size                  object
p_last                 float64
p_delta                float64
p_gamma                float64
p_vega                 float64
p_theta                float64
p_rho                  float64
p_iv                   float64
p_volume                object
strike_distance        float64
strike_d

In [17]:
test_data = data.head(10)

In [18]:
data.to_csv('data/cleaned_spy_eod_202312.csv', index=False)
test_data.to_csv('data/test_data.csv', index=False)